In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import skimage.io

import os.path

import sys
sys.path.insert(0, '/home/jr0th/github/segmentation/code/')
import helper.metrics

import numpy as np

In [ ]:
dl_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/DL_labels/'
gt_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/GT_labels/'

mat_dl_dir = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/IoU_DL/'

In [ ]:
labels_dl = skimage.io.imread_collection(dl_label + "*.png")

# buffer for results
results = np.empty(shape = (0, 4))

# loop over all images
for index in range(len(labels_dl)):
    
    # print progress
    print(index + 1, '/', len(labels_dl))

    path = labels_dl.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    # get labels
    label_dl = labels_dl[index]
    label_gt = skimage.io.imread(gt_label + filename)

    nb_gt_cells = np.max(label_gt)
    
    [nb_overdetection, nb_underdetection, mean_IoU, IoUs] = helper.metrics.compare_two_labels(label_dl, label_gt, True)
    
    np.save(mat_dl_dir + filename_wo_ext, IoUs)
    
    result = np.hstack((nb_overdetection, nb_underdetection, mean_IoU, nb_gt_cells))
    results = np.vstack((results, result))

print(results)

# calculate error ratio
sums = np.sum(results, axis = 0)
nb_gt_cells_total = sums[3]
nb_errors = sums[0] + sums[1]

err_ratio = nb_errors / nb_gt_cells_total
print(err_ratio)

np.savetxt('./results_table_DL.txt', results)
np.savetxt('./results_summary_DL.txt', np.array([nb_errors, nb_gt_cells_total, err_ratio]))